The Dataset contains the following 12 features:

CustomerID: A unique identifier

Age: The age of the customer

Gender: The gender of the customer

Tenure: The number of months the customer has stayed with the company

Usage Frequency: The number of times the customer has used the service the past month

Support calls: The number of support calls the customer has made the past month

Payment Delay: Number of days the customer has delayed payment the past month

Subscription Type: The type of subscription the customer has

Contract Length: Duration of the contract

Total Spend: The total amount the customer has spent

Last Interaction: Number of days since the last interaction the customer has had with the company

Churn: Whether the customer has churned or not

# Import packages

In [ ]:
import winsound
import itertools
import pandas as pd
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Cleanup

In [ ]:
test_set_dirty = pd.read_csv("Datasets/In/customer_churn_dataset-testing-master.csv", sep=",")
training_set_dirty = pd.read_csv("Datasets/In/customer_churn_dataset-training-master.csv", sep=",")

combined_set_dirty = pd.concat([training_set_dirty, test_set_dirty], ignore_index=True)
combined_set_dirty = combined_set_dirty.drop(combined_set_dirty.columns[0], axis=1)

In [ ]:
missing_values = combined_set_dirty.isnull().sum()
missing_values

In [ ]:
combined_set_dirty[combined_set_dirty.isna().any(axis=1)]

In [ ]:
combined_set_dirty.dropna(inplace=True)

combined_set_dirty.columns = [col.lower().replace(" ", "_") for col in combined_set_dirty.columns]
combined_set_dirty.info()

In [ ]:
combined_set_dirty[combined_set_dirty.isna().any(axis=1)]

In [ ]:
numerals = ["age", "tenure", "usage_frequency", "support_calls", "payment_delay", "last_interaction", "churn"]

for col in numerals:
    combined_set_dirty[col] = combined_set_dirty[col].astype(int)

In [ ]:
cleaned_set = combined_set_dirty.copy()

# Descriptive Analytics

In [ ]:
# Summary statistics
print("Summary Statistics for Churned Customers:")
print(cleaned_set[cleaned_set['churn'] == 1].describe())
print("\nSummary Statistics for Non-Churned Customers:")
print(cleaned_set[cleaned_set['churn'] == 0].describe())

# Correlation analysis
correlation = cleaned_set.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='crest')
plt.title('Correlation Heatmap')
plt.show()

# Churn rate
churn_rate = cleaned_set['churn'].mean() * 100
print(f"Churn Rate: {churn_rate}%")

In [ ]:
numeric_cols = ["age", "tenure", "usage_frequency", "support_calls", "payment_delay", "last_interaction", "total_spend"]

num_bins = 3

excourse_set = cleaned_set.copy()

for col in numeric_cols:
    excourse_set[col] = pd.cut(cleaned_set[col], num_bins, duplicates='drop')
    print(col)
    for interval in excourse_set[col].cat.categories:
        print(interval)

In [ ]:
columns = ["age", "gender", "tenure", "usage_frequency", "support_calls", "payment_delay", "subscription_type",
           "contract_length", "total_spend", "last_interaction"]

stacked_data_percent = {}

for col in columns:
    category_counts = excourse_set.groupby([col, "churn"]).size().unstack(fill_value=0)

    category_percent = category_counts.div(category_counts.sum(axis=1), axis=0) * 100
    print(category_percent)
    stacked_data_percent[col] = category_percent

In [ ]:
# Calculate the overall churn rate
overall_churn_rate = excourse_set['churn'].mean() * 100

# Add a new row to each DataFrame in stacked_data_percent with the overall churn rate
for col, df in stacked_data_percent.items():
    df.loc['Overall'] = [100 - overall_churn_rate, overall_churn_rate]

colors = {0: 'green', 1: 'red'}
for col, df in stacked_data_percent.items():
    ax = df.plot(kind='barh', stacked=True, color=[colors[churn] for churn in df.columns],
                 title=f'Percentage Chart of Churned Customers in {col}')
    plt.ylabel(col)
    plt.xlabel('Percentage')
    plt.legend(["No Churn", "Churn"], loc='best')

    # Add the percentage values on each bar
    for p in ax.patches:
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        ax.text(x + width / 2,
                y + height / 2,
                '{:.1f} %'.format(width),
                horizontalalignment='center',
                verticalalignment='center')
    plt.show()

In [ ]:
combinations = list(itertools.combinations(columns, 2))

# Analyze each combination
for combination in combinations:
    # Create a multi-index DataFrame
    multi_index_df = excourse_set.set_index(list(combination) + ['churn'])

    # Calculate the size of each group
    grouped_df = multi_index_df.groupby(list(combination) + ['churn']).size()

    # Unstack the DataFrame to get a cross-tabulation
    cross_tab = grouped_df.unstack(fill_value=0)

    # Convert absolute numbers to relative percentages
    cross_tab_percent = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100

    # Print the cross-tabulation
    print(f"Cross-tabulation for {combination}:")
    print(cross_tab_percent)
    print("\n")

In [ ]:
combinations = list(itertools.combinations(columns, 3))

# Analyze each combination
for combination in combinations:
    # Create a multi-index DataFrame
    multi_index_df = excourse_set.set_index(list(combination) + ['churn'])

    # Calculate the size of each group
    grouped_df = multi_index_df.groupby(list(combination) + ['churn']).size()

    # Unstack the DataFrame to get a cross-tabulation
    cross_tab = grouped_df.unstack(fill_value=0)

    # Convert absolute numbers to relative percentages
    cross_tab_percent = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100

    # Print the cross-tabulation
    print(f"Cross-tabulation for {combination}:")
    print(cross_tab_percent)
    print("\n") 

# Predictive Analytics

In [ ]:
prepared_set = cleaned_set.copy()

# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse_output=False)

# Identify categorical columns
categorical_cols = ['gender', 'subscription_type', 'contract_length']

# Fit and transform the data, converting it into a DataFrame
prepared_set_encoded = pd.DataFrame(encoder.fit_transform(prepared_set[categorical_cols]))

# Get feature names from the encoder and assign them as column names
prepared_set_encoded.columns = encoder.get_feature_names_out(categorical_cols)
 
# Drop the original categorical columns
prepared_set.drop(categorical_cols, axis=1, inplace=True)

# Reset the indices of the dataframes
prepared_set = prepared_set.reset_index(drop=True)
prepared_set_encoded = prepared_set_encoded.reset_index(drop=True)

# Concatenate the original DataFrame with the one-hot encoded DataFrame
prepared_set = pd.concat([prepared_set, prepared_set_encoded], axis=1)

prepared_set.info()

## Primitive Approach

In [ ]:
primitive_set = prepared_set.copy()

X = primitive_set.drop('churn', axis=1)
y = primitive_set['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = RidgeClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
primitive_mse = mean_squared_error(y_test, y_pred)
primitive_mae = mean_absolute_error(y_test, y_pred)
primitive_r2 = r2_score(y_test, y_pred)
primitive_accuracy = accuracy_score(y_test, y_pred)
primitive_precision = precision_score(y_test, y_pred)
primitive_recall = recall_score(y_test, y_pred)
primitive_f1 = f1_score(y_test, y_pred)
primitive_roc_auc = roc_auc_score(y_test, y_pred)

print(
    f"MSE: {primitive_mse}\nMAE: {primitive_mae}\nR2: {primitive_r2}\nAccuracy: {primitive_accuracy}\nPrecision: {primitive_precision}\nRecall: {primitive_recall}\nF1 Score: {primitive_f1}\nROC AUC: {primitive_roc_auc}")

# My Model
## Define Target and Feauture Variables & Split and Scale Set
Split the data into training, validation, and test sets, then, standardise the features

In [ ]:
better_model = prepared_set.copy()

# Define Target and feature variables
y = better_model['churn'].values
X = better_model.drop(['churn'], axis=1)

# Extract feature names
feature_names = X.columns.tolist()

# Perform train-validation-test split
X_train_val, X_test, y_train_val, y_test = train_test_split(X[feature_names], y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.285, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_train_val = scaler.transform(X_train_val)
X_test = scaler.transform(X_test)

In [ ]:
def beep():
    duration = 5000
    freq = 1000
    winsound.Beep(freq, duration)

### Train and Evaluate Models

Perform hyperparameter tuning for the RandomForest model using randomised search. Print the best hyperparameters and the corresponding Recall(on subset of training set(Cross-Validation)) score.

In [ ]:
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

rf_model = RandomForestClassifier(random_state=42)

rf_random = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=100, cv=3, verbose=2,random_state=42, n_jobs=-1, scoring='recall')

rf_random.fit(X_train, y_train)

print(rf_random.best_score_) # 0.9959059371040283
print(rf_random.best_params_) # {'n_estimators': 1800, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 80, 'bootstrap': False}
# Runtime 2h 51m 14s
beep()

Perform hyperparameter tuning for the XGBoost model using randomised search. Print the best hyperparameters and the corresponding Recall(on subset of training set(Cross-Validation)) score.

In [ ]:
param_grid = {
    'learning_rate': [0.05, 0.1, 0.20, 0.30],
    'n_estimators': [100, 400, 800],
    'max_depth': [3, 6, 9],
    'min_child_weight': [1, 10, 100],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'subsample': [0.5, 0.7, 1.0]
}

xgb_model = xgb.XGBClassifier(random_state=42)

xgb_random = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, n_iter=300, cv=3, verbose=2, n_jobs=-1, scoring='recall')

xgb_random.fit(X_train, y_train)

print(xgb_random.best_score_) # 0.9973181753786369
print(xgb_random.best_params_) # {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
# Runtime 1h 47m 7s
beep()

Perform hyperparameter tuning for the LightGBM using grid search. Print the best hyperparameters and the corresponding Recall(on subset of training set(Cross-Validation)) score.

In [ ]:
# Define the parameter lgbm_grid
param_grid = {
    'max_depth': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 1],
    'num_leaves': [31, 62, 93],
    'n_estimators': [100, 200, 300]
}

# Create a LightGBM model
lgbm_model = lgb.LGBMClassifier()

# Create the lgbm_grid search object
lgbm_grid = GridSearchCV(lgbm_model, param_grid, cv=5, scoring='recall')

# Fit the lgbm_grid search object to the data
lgbm_grid.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print(lgbm_grid.best_params_) # {'learning_rate': 0.1, 'max_depth': 30, 'n_estimators': 100, 'num_leaves': 93}
print(lgbm_grid.best_score_) # 0.9991155677841
# Runtime 8m 31s 633ms
beep()

Perform hyperparameter tuning for the Decision Tree model using grid search. Print the best hyperparameters and the corresponding Recall(on subset of training set(Cross-Validation)) score

In [ ]:
# Define the parameter tree_grid
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [1.0, 'sqrt', 'log2', None]
}

# Create a DecisionTreeRegressor model
decTree_model = DecisionTreeClassifier()

#Create the tree_grid search object
tree_grid = GridSearchCV(decTree_model, param_grid, cv=5, scoring="recall", verbose=1)

# fit the tree_grid search object to the data
tree_grid.fit(X_train, y_train)

#Print the best parameters and the corresponding score
print(tree_grid.best_params_)  # {'max_depth': 10, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 2}
print(tree_grid.best_score_)  # 0.9912056159258545
# Runtime 7m 22s 647ms
beep()

In [ ]:
y_pred_rf = rf_random.best_estimator_.predict(X_val)
y_pred_xgb = xgb_random.best_estimator_.predict(X_val)
y_pred_lgbm = lgbm_grid.best_estimator_.predict(X_val)
y_pred_tree = tree_grid.best_estimator_.predict(X_val)

In [ ]:
#Accuracy, Precision, Recall, F1 Score and ROC AUC
# Calculate the performance metrics
rf_accuracy = accuracy_score(y_val, y_pred_rf)
rf_precision = precision_score(y_val, y_pred_rf)
rf_recall = recall_score(y_val, y_pred_rf)
rf_f1 = f1_score(y_val, y_pred_rf)
rf_roc_auc = roc_auc_score(y_val, y_pred_rf)

xgb_accuracy = accuracy_score(y_val, y_pred_xgb)
xgb_precision = precision_score(y_val, y_pred_xgb)
xgb_recall = recall_score(y_val, y_pred_xgb)
xgb_f1 = f1_score(y_val, y_pred_xgb)
xgb_roc_auc = roc_auc_score(y_val, y_pred_xgb)

lgbm_accuracy = accuracy_score(y_val, y_pred_lgbm)
lgbm_precision = precision_score(y_val, y_pred_lgbm)
lgbm_recall = recall_score(y_val, y_pred_lgbm)
lgbm_f1 = f1_score(y_val, y_pred_lgbm)
lgbm_roc_auc = roc_auc_score(y_val, y_pred_lgbm)

tree_accuracy = accuracy_score(y_val, y_pred_tree)
tree_precision = precision_score(y_val, y_pred_tree)
tree_recall = recall_score(y_val, y_pred_tree)
tree_f1 = f1_score(y_val, y_pred_tree)
tree_roc_auc = roc_auc_score(y_val, y_pred_tree)

models = ['Random Forest (Validation Set)', 'XGBoost (Validation Set)', 'LightGBM (Validation Set)', 'Decision Tree (Validation Set)']
accuracy_values = [rf_accuracy, xgb_accuracy, lgbm_accuracy, tree_accuracy]
precision_values = [rf_precision, xgb_precision, lgbm_precision, tree_precision]
recall_values = [rf_recall, xgb_recall, lgbm_recall, tree_recall]
f1_values = [rf_f1, xgb_f1, lgbm_f1, tree_f1]
roc_auc_values = [rf_roc_auc, xgb_roc_auc, lgbm_roc_auc, tree_roc_auc]

In [ ]:
def print_values(values, models, title):
    print(title)
    for model, value in zip(models, values):
        print(f"{model}: {value}")

def plot_values(values, title, models, ylabel):
    plt.figure(figsize=(15, 6))
    sns.barplot(x=models, y=values)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.show()

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(title)
    plt.show()

In [ ]:
print_values(accuracy_values, models, 'Comparison of Accuracy')
print_values(precision_values, models, 'Comparison of Precision')
print_values(recall_values, models, 'Comparison of Recall')
print_values(f1_values, models, 'Comparison of F1 Score')
print_values(roc_auc_values, models, 'Comparison of ROC AUC')

In [ ]:
plot_values(accuracy_values, 'Comparison of Accuracy', models, 'Accuracy')
plot_values(precision_values, 'Comparison of Precision', models, 'Precision')
plot_values(recall_values, 'Comparison of Recall', models, 'Recall')
plot_values(f1_values, 'Comparison of F1 Score', models, 'F1 Score')
plot_values(roc_auc_values, 'Comparison of ROC AUC', models, 'ROC AUC')

In [ ]:
plot_confusion_matrix(y_val, y_pred_rf, 'Random Forest Confusion Matrix')
plot_confusion_matrix(y_val, y_pred_xgb, 'XGBoost Confusion Matrix')
plot_confusion_matrix(y_val, y_pred_lgbm, 'LightGBM Confusion Matrix')
plot_confusion_matrix(y_val, y_pred_tree, 'Decision Tree Confusion Matrix')

Implement the Stacking Classifier using the best models and hyperparameters obtained from the previous steps.

In [ ]:
rf_best_params = {'n_estimators': 1800, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 80, 'bootstrap': False}
xgb_best_params = {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
lgbm_best_params = {'learning_rate': 0.1, 'max_depth': 30, 'n_estimators': 100, 'num_leaves': 93}
tree_best_params = {'max_depth': 10, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 2}

base_models = [
    ('Random Forest', RandomForestClassifier(**rf_best_params)),
    ('XGBoost', xgb.XGBClassifier(**xgb_best_params)),
    ('LightGBM', lgb.LGBMClassifier(**lgbm_best_params)),
    ('Decision Tree', DecisionTreeClassifier(**tree_best_params))
]

meta_model = lgb.LGBMClassifier()

stacked_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, verbose=2)

In [ ]:
stacked_model.fit(X_train, y_train)
beep()

In [ ]:
rf_model = RandomForestClassifier(**rf_best_params)
rf_model.fit(X_train, y_train)
beep()

In [ ]:
xgb_model = xgb.XGBClassifier(**xgb_best_params)
xgb_model.fit(X_train, y_train)
beep()

In [ ]:
lgbm_model = lgb.LGBMClassifier(**lgbm_best_params)
lgbm_model.fit(X_train, y_train)
beep()

In [ ]:
decTree_model = DecisionTreeClassifier(**tree_best_params)
decTree_model.fit(X_train, y_train)
beep()

# Evaluate all the models on the validation set

In [ ]:
# Predict validation set and calculate performance matrices
y_val_pred_rf = rf_model.predict(X_val)
y_val_pred_xgb = xgb_model.predict(X_val)
y_val_pred_lgbm = lgbm_model.predict(X_val)
y_val_pred_tree = decTree_model.predict(X_val)
y_val_stack = stacked_model.predict(X_val)

# Calculate the performance metrics
stack_accuracy = accuracy_score(y_val, y_val_stack)
stack_precision = precision_score(y_val, y_val_stack)
stack_recall = recall_score(y_val, y_val_stack)
stack_f1 = f1_score(y_val, y_val_stack)
stack_roc_auc = roc_auc_score(y_val, y_val_stack)

rf_accuracy = accuracy_score(y_val, y_val_pred_rf)
rf_precision = precision_score(y_val, y_val_pred_rf)
rf_recall = recall_score(y_val, y_val_pred_rf)
rf_f1 = f1_score(y_val, y_val_pred_rf)
rf_roc_auc = roc_auc_score(y_val, y_val_pred_rf)

xgb_accuracy = accuracy_score(y_val, y_val_pred_xgb)
xgb_precision = precision_score(y_val, y_val_pred_xgb)
xgb_recall = recall_score(y_val, y_val_pred_xgb)
xgb_f1 = f1_score(y_val, y_val_pred_xgb)
xgb_roc_auc = roc_auc_score(y_val, y_val_pred_xgb)

lgbm_accuracy = accuracy_score(y_val, y_val_pred_lgbm)
lgbm_precision = precision_score(y_val, y_val_pred_lgbm)
lgbm_recall = recall_score(y_val, y_val_pred_lgbm)
lgbm_f1 = f1_score(y_val, y_val_pred_lgbm)
lgbm_roc_auc = roc_auc_score(y_val, y_val_pred_lgbm)

tree_accuracy = accuracy_score(y_val, y_val_pred_tree)
tree_precision = precision_score(y_val, y_val_pred_tree)
tree_recall = recall_score(y_val, y_val_pred_tree)
tree_f1 = f1_score(y_val, y_val_pred_tree)
tree_roc_auc = roc_auc_score(y_val, y_val_pred_tree)

models = ['Stacking (Validation Set)', 'Random Forest (Validation Set)', 'XGBoost (Validation Set)', 'LightGBM (Validation Set)', 'Decision Tree (Validation Set)']
accuracy_values = [stack_accuracy, rf_accuracy, xgb_accuracy, lgbm_accuracy, tree_accuracy]
precision_values = [stack_precision, rf_precision, xgb_precision, lgbm_precision, tree_precision]
recall_values = [stack_recall, rf_recall, xgb_recall, lgbm_recall, tree_recall]
f1_values = [stack_f1, rf_f1, xgb_f1, lgbm_f1, tree_f1]
roc_auc_values = [stack_roc_auc, rf_roc_auc, xgb_roc_auc, lgbm_roc_auc, tree_roc_auc]

In [ ]:
print_values(accuracy_values, models, 'Comparison of Accuracy')
print_values(precision_values, models, 'Comparison of Precision')
print_values(recall_values, models, 'Comparison of Recall')
print_values(f1_values, models, 'Comparison of F1 Score')
print_values(roc_auc_values, models, 'Comparison of ROC AUC')

In [ ]:
plot_confusion_matrix(y_val, y_val_stack, 'Stacking Confusion Matrix (Validation Set)')
plot_confusion_matrix(y_val, y_val_pred_rf, 'Random Forest Confusion Matrix (Validation Set)')
plot_confusion_matrix(y_val, y_val_pred_xgb, 'XGBoost Confusion Matrix (Validation Set)')
plot_confusion_matrix(y_val, y_val_pred_lgbm, 'LightGBM Confusion Matrix (Validation Set)')
plot_confusion_matrix(y_val, y_val_pred_tree, 'Decision Tree Confusion Matrix (Validation Set)')

# Evaluate the models on the test set

In [ ]:
# Predict test set and calculate performance matrices
y_test_pred_rf = rf_model.predict(X_test)
y_test_pred_xgb = xgb_model.predict(X_test)
y_test_pred_lgbm = lgbm_model.predict(X_test)
y_test_pred_tree = decTree_model.predict(X_test)
y_test_stack = stacked_model.predict(X_test)

# Calculate the performance metrics
stack_accuracy = accuracy_score(y_test, y_test_stack)
stack_precision = precision_score(y_test, y_test_stack)
stack_recall = recall_score(y_test, y_test_stack)
stack_f1 = f1_score(y_test, y_test_stack)
stack_roc_auc = roc_auc_score(y_test, y_test_stack)

rf_accuracy = accuracy_score(y_test, y_test_pred_rf)
rf_precision = precision_score(y_test, y_test_pred_rf)
rf_recall = recall_score(y_test, y_test_pred_rf)
rf_f1 = f1_score(y_test, y_test_pred_rf)
rf_roc_auc = roc_auc_score(y_test, y_test_pred_rf)

xgb_accuracy = accuracy_score(y_test, y_test_pred_xgb)
xgb_precision = precision_score(y_test, y_test_pred_xgb)
xgb_recall = recall_score(y_test, y_test_pred_xgb)
xgb_f1 = f1_score(y_test, y_test_pred_xgb)
xgb_roc_auc = roc_auc_score(y_test, y_test_pred_xgb)

lgbm_accuracy = accuracy_score(y_test, y_test_pred_lgbm)
lgbm_precision = precision_score(y_test, y_test_pred_lgbm)
lgbm_recall = recall_score(y_test, y_test_pred_lgbm)
lgbm_f1 = f1_score(y_test, y_test_pred_lgbm)
lgbm_roc_auc = roc_auc_score(y_test, y_test_pred_lgbm)

tree_accuracy = accuracy_score(y_test, y_test_pred_tree)
tree_precision = precision_score(y_test, y_test_pred_tree)
tree_recall = recall_score(y_test, y_test_pred_tree)
tree_f1 = f1_score(y_test, y_test_pred_tree)
tree_roc_auc = roc_auc_score(y_test, y_test_pred_tree)

models = ['Stacking (Test Set)', 'Random Forest (Test Set)', 'XGBoost (Test Set)', 'LightGBM (Test Set)', 'Decision Tree (Test Set)']
accuracy_values = [stack_accuracy, rf_accuracy, xgb_accuracy, lgbm_accuracy, tree_accuracy]
precision_values = [stack_precision, rf_precision, xgb_precision, lgbm_precision, tree_precision]
recall_values = [stack_recall, rf_recall, xgb_recall, lgbm_recall, tree_recall]
f1_values = [stack_f1, rf_f1, xgb_f1, lgbm_f1, tree_f1]
roc_auc_values = [stack_roc_auc, rf_roc_auc, xgb_roc_auc, lgbm_roc_auc, tree_roc_auc]

In [ ]:
print_values(accuracy_values, models, 'Comparison of Accuracy')
print_values(precision_values, models, 'Comparison of Precision')
print_values(recall_values, models, 'Comparison of Recall')
print_values(f1_values, models, 'Comparison of F1 Score')
print_values(roc_auc_values, models, 'Comparison of ROC AUC')

In [ ]:
plot_confusion_matrix(y_test, y_test_stack, 'Stacking Confusion Matrix (Test Set)')
plot_confusion_matrix(y_test, y_test_pred_rf, 'Random Forest Confusion Matrix (Test Set)')
plot_confusion_matrix(y_test, y_test_pred_xgb, 'XGBoost Confusion Matrix (Test Set)')
plot_confusion_matrix(y_test, y_test_pred_lgbm, 'LightGBM Confusion Matrix (Test Set)')
plot_confusion_matrix(y_test, y_test_pred_tree, 'Decision Tree Confusion Matrix (Test Set)')

# 2nd Attempt with best params from base model for meta model

In [ ]:
meta_model = lgb.LGBMClassifier(**lgbm_best_params)
second_stacked_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, verbose=2)

In [ ]:
second_stacked_model.fit(X_train, y_train)
beep()

In [ ]:
# Predict validation set and calculate performance matrices
y_val_stack = stacked_model.predict(X_val)
second_y_val_stack = second_stacked_model.predict(X_val)

y_test_stack = stacked_model.predict(X_test)
second_y_test_stack = second_stacked_model.predict(X_test)

# Calculate the performance metrics
y_val_stack_accuracy = accuracy_score(y_val, y_val_stack)
y_val_stack_precision = precision_score(y_val, y_val_stack)
y_val_stack_recall = recall_score(y_val, y_val_stack)
y_val_stack_f1 = f1_score(y_val, y_val_stack)
y_val_stack_roc_auc = roc_auc_score(y_val, y_val_stack)

second_val_stack_accuracy = accuracy_score(y_val, second_y_val_stack)
second_val_stack_precision = precision_score(y_val, second_y_val_stack)
second_val_stack_recall = recall_score(y_val, second_y_val_stack)
second_val_stack_f1 = f1_score(y_val, second_y_val_stack)
second_val_stack_roc_auc = roc_auc_score(y_val, second_y_val_stack)


y_test_stack_accuracy = accuracy_score(y_test, y_test_stack)
y_test_stack_precision = precision_score(y_test, y_test_stack)
y_test_stack_recall = recall_score(y_test, y_test_stack)
y_test_stack_f1 = f1_score(y_test, y_test_stack)
y_test_stack_roc_auc = roc_auc_score(y_test, y_test_stack)

second_test_stack_accuracy = accuracy_score(y_test, second_y_test_stack)
second_test_stack_precision = precision_score(y_test, second_y_test_stack)
second_test_stack_recall = recall_score(y_test, second_y_test_stack)
second_test_stack_f1 = f1_score(y_test, second_y_test_stack)
second_test_stack_roc_auc = roc_auc_score(y_test, second_y_test_stack)

models = ['Stacking (Validation Set)', 'Stacking (Test Set)', '2nd Stacking (Validation Set)', '2nd Stacking (Test Set)']
accuracy_values = [y_val_stack_accuracy, y_test_stack_accuracy, second_val_stack_accuracy, second_test_stack_accuracy]
precision_values = [y_val_stack_precision, y_test_stack_precision, second_val_stack_precision, second_test_stack_precision]
recall_values = [y_val_stack_recall, y_test_stack_recall, second_val_stack_recall, second_test_stack_recall]
f1_values = [y_val_stack_f1, y_test_stack_f1, second_val_stack_f1, second_test_stack_f1]
roc_auc_values = [y_val_stack_roc_auc, y_test_stack_roc_auc, second_val_stack_roc_auc, second_test_stack_roc_auc]

print_values(accuracy_values, models, 'Comparison of Accuracy')
print_values(precision_values, models, 'Comparison of Precision')
print_values(recall_values, models, 'Comparison of Recall')
print_values(f1_values, models, 'Comparison of F1 Score')
print_values(roc_auc_values, models, 'Comparison of ROC AUC')

plot_confusion_matrix(y_val, y_val_stack, 'Stacking Confusion Matrix (Validation Set)')
plot_confusion_matrix(y_val, second_y_val_stack, '2nd Stacking Confusion Matrix (Validation Set)')
plot_confusion_matrix(y_test, y_test_stack, 'Stacking Confusion Matrix (Test Set)')
plot_confusion_matrix(y_test, second_y_test_stack, '2nd Stacking Confusion Matrix (Test Set)')

In [ ]:
beep()